In [133]:
from collections import Counter
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import classification_report
import textdistance
from string import punctuation
from tqdm import tqdm
punct = set(punctuation)

In [123]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)
    
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [76]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

Сначала отбираем 3 ближайших по косинусному расстроянию слов, затем из них выбираем ближайшее по левинштейну.

In [108]:
def get_closest_hybrid_match(text, X, vec):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:3]
    lookup = [id2word[top] for top in topn]
    similarities = Counter()
    for word in lookup:
        similarities[word] = textdistance.levenshtein.normalized_similarity(text, word)
    closest = 0
    return similarities.most_common(1)[0][0]

Работает очень даже быстро:

In [109]:
%%time
get_closest_hybrid_match('апофеоз',  X, vec)

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 36.2 ms


'апофеоз'

In [100]:
y_true = []
y_pred = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    pairs_predict = []
    for pair in word_pairs:
        pairs_predict.append([pair[0], get_closest_hybrid_match(pair[1],  X, vec)])
    for pair in pairs_predict:
        y_true.append(1)
        if pair[0] == pair[1]:
            y_pred.append(1)
        else:
            y_pred.append(0)

100%|██████████| 916/916 [04:21<00:00,  3.10it/s]


In [101]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.79      0.89     10012

   micro avg       0.79      0.79      0.79     10012
   macro avg       0.50      0.40      0.44     10012
weighted avg       1.00      0.79      0.89     10012



Качество где-то хромает, посмотрим на ошибки.

In [115]:
df_dict = {'true':[], 'bad':[], 'correction':[]}

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] != get_closest_hybrid_match(pair[1],  X, vec):
            df_dict['true'].append(pair[0])
            df_dict['bad'].append(pair[1])
            df_dict['correction'].append(get_closest_hybrid_match(pair[1],  X, vec))

100%|██████████| 916/916 [05:04<00:00,  2.64it/s]


In [114]:
df = pd.DataFrame(df_dict)
df.head(20)

,true,bad,correction
0,симпатичнейшее,симпатичнейшое,пластичнейшими
1,шпионское,шпионское,шпионские
2,гламурный,гламурный,гуманный
3,бонда,бонда,банд
4,superheadz,superheadz,herederas
5,clap,clap,place
6,camera,camera,america
7,получатся,полчатся,ополчатся
8,язычки,язычки,язычка
9,все,все,есв


Проблема в том, что, видимо, векторно блишайшее слово и эталон без опечатки иногда не совпадают, поэтому алгоритм исправляет то, не стоило бы. Качество можно улучшить, добавив подкрепление корпусом - если слово до исправления в нем есть, значит, скорее всего, это слово без опечатки.

In [137]:
grand_corpus = []
for i in range(len(corpus)):
    for word in corpus[i]:
        grand_corpus.append(word)
print(len(grand_corpus))

1664313


In [141]:
y_true = []
y_pred = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    pairs_predict = []
    for pair in word_pairs:
        if pair[1] not in grand_corpus:
            pairs_predict.append([pair[0], get_closest_hybrid_match(pair[1],  X, vec)])
        else:
            pairs_predict.append([pair[0], pair[1]])
    for pair in pairs_predict:
        y_true.append(1)
        if pair[0] == pair[1]:
            y_pred.append(1)
        else:
            y_pred.append(0)

100%|██████████| 916/916 [01:55<00:00,  7.92it/s]


In [142]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.83      0.90     10012

   micro avg       0.83      0.83      0.83     10012
   macro avg       0.50      0.41      0.45     10012
weighted avg       1.00      0.83      0.90     10012



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Качество улучшилось (хотя и не ахти, я думаю, из-за того, что корпус маловат), и работaет это более чем в полтора раза быстрее.